In [1]:
import csv, io, html
import plotly.graph_objs as go

wgi = '''Year;Political Party;Political Orientation;Control of Corruption;Rule of Law;Regulatory Quality;Government Effectiveness;Political Stability No Violence;Voiceand Accountability
1996;Ulivo;Center-Left;67,2;84,42;76,09;78,14;86,7;84,5
1998;Ulivo;Center-Left;68,45;73;72,28;78,69;90,96;79,6
2000;Ulivo;Center-Left;75;73,63;76,63;77,05;78,31;79,6
2002;Forza Italia;Center-Right;72,49;70,65;75,68;74,59;75,66;78,11
2003;Forza Italia;Center-Right;70,9;69,65;80;75,68;59,8;77,61
2004;Forza Italia;Center-Right;66,5;68,27;81,09;73,13;54,85;86,54
2005;Forza Italia;Center-Right;67,8;64,59;78,43;69,61;63,11;81,73
2006;Forza Italia / Unione;Center-Right / Center-Left;71,22;61,72;78,92;64,88;62,32;83,17
2007;Unione;Center-Left;67,48;63,64;78,16;58,74;60,39;86,06
2008;Unione / Forza Italia;Center-Left / Center-Right;64,08;62,5;79,13;62,62;65,38;81,25
2009;Forza Italia;Center-Right;64,11;62,56;78,47;66,99;54,5;81,52
2010;Forza Italia;Center-Right;64,29;63,98;77,51;67,46;62,56;75,83
2011;Forza Italia;Center-Right;65,88;64,32;74,88;65,88;62,56;75,12
2012;Tecnico Monti;Centro;62,09;64,32;74,41;66,35;63,03;74,65
2013;Tecnico Monti / PD;Centro / Center-Left;60,19;63,85;74,88;68,72;63,98;76,06
2014;PD;Center-Left;56,73;67,79;71,15;70,67;60,48;77,83
2015;PD;Center-Left;57,14;63,33;73,81;70,48;58,57;81,28
2016;PD;Center-Left;59,05;62,86;73,81;71,9;58,57;81,77
2017;PD;Center-Left;60,48;62,38;74,76;69,05;57,62;78,82
2018;PD / M5S e Lega;Center-Left / Center-Right;60,95;60;75,24;67,62;57,08;78,16
2019;M5S e Lega / M5S e PD;Center-Right / Center-Left;61,43;60,48;77,14;68,57;58,96;76,81
2020;M5S e PD;Center-Left;67,14;57,62;68,1;65,24;58,02;81,16
2021;Tecnico Draghi;Centro;67,62;58,1;68,57;64,29;63,68;83,57
2022;Tecnico Draghi;Centro;68,87;58,49;68,87;66,98;58,49;82,61'''

party_colors = {
	'Center-Left': '#ff0000',
	'Center-Right': '#0000ff',
	'Center': '#666',
}

wgi = io.StringIO(wgi)
reader = csv.reader(wgi, delimiter=';')
table = [row for row in reader]
table_traspose = [list(row) for row in zip(*table)]

# Estrai le etichette per l'asse x e i dati per il grafico
anno = table_traspose[0][1:]
partito = table_traspose[1][1:]
x_labels = [f"{a} {p}" for a, p in zip(anno, partito)]
data = []

# Aggiungi le linee per ogni indicatore
for i in range(3, len(table_traspose)):
	column_value = [float(s.replace(',', '.')) for s in table_traspose[i][1:]]
	data.append(go.Scatter(
		x=list(range(len(x_labels))),  # Usa indici numerici
		y=column_value,
		mode='lines+markers',
		name=html.escape(table_traspose[i][0])  # Gestisci i caratteri speciali
	))

# Aggiungi le bande verticali colorate per i partiti
shapes = []
for i in range(len(x_labels)):
	try:
		partito = table_traspose[2][i + 1]
		if "/" in partito:
			cnf = [[-1, -0.5, partito.split('/')[0]], [-0.5, 0, partito.split('/')[1]]]
		else:
			cnf = [[-1, 0, partito]]

		for c in cnf:
			color = party_colors.get(c[2].strip(), 'gray')
			shapes.append(dict(
				type="rect",
				xref="x",
				yref="paper",
				x0=i + c[0],
				x1=i + c[1],
				y0=0,
				y1=1,
				fillcolor=color,
				opacity=0.2,
				layer="below",
				line_width=0,
			))
	except:
		pass

# Crea il layout del grafico con le bande colorate e la legenda personalizzata
layout = go.Layout(
	title='Worldwide Governance Indicators in Italy',
	xaxis=dict(
		title='Year and Party',
		tickvals=list(range(len(x_labels))),  # Allinea i tick con gli indici numerici
		ticktext=x_labels,  # Mostra le etichette originali
		tickangle=45,
		gridcolor='lightgrey'  # Colore dello sfondo dell'asse x
	),
	plot_bgcolor='white',  # Colore dello sfondo dell'area del grafico
	paper_bgcolor='white',  # Colore dello sfondo dell'intera figura
	shapes=shapes,
	hovermode='closest',
)

# Crea la figura e mostra il grafico interattivo
fig = go.Figure(data=data, layout=layout)
fig.show()